<a href="https://colab.research.google.com/github/reynierhdez/py-mpca-ann/blob/master/mpcaANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import mpi4py
import numpy as np
import pandas as pd

In [0]:
import random

class BaseAlgorithm(object):
    def __init__(self, function_wrapper, number_of_variables = 1, objective = "minimization"):
        self.function_wrapper = function_wrapper
        self.number_of_variables = number_of_variables
        self.objective = objective

    def get_decision_variable_value_by_randomization(self, decision_variable_index):
        return self.function_wrapper.minimum_decision_variable_values()[decision_variable_index] + (self.function_wrapper.maximum_decision_variable_values()[decision_variable_index] - self.function_wrapper.minimum_decision_variable_values()[decision_variable_index]) * random.random()

    def best_function_value_from_list(self, function_values):
        if self.objective == "maximization":
            best_function_value = max(function_values)
        elif self.objective == "minimization":
            best_function_value = min(function_values)

        return best_function_value

In [0]:
class MultiParticleCollisionAlgorithm(BaseAlgorithm):

    def __init__(self, function_wrapper, number_of_variables = 1, objective = "minimization"):
        super().__init__(function_wrapper, number_of_variables, objective)

    def search(self, number_of_particles = 20, number_of_iterations = 100, social_coefficient = 0.5, random_variable_coefficient = 0.2):

        self.__initialize_particles(number_of_particles)

        global_best_position = None
        best_function_value = None

        for iteration in range(number_of_iterations):

            function_values = [self.function_wrapper.objective_function_value(self.__particle_locations[particle_location_index]) for particle_location_index in range(len(self.__particle_locations))]

            best_function_value = super(MultiParticleCollisionAlgorithm, self).best_function_value_from_list(function_values)
            global_best_position = self.__particle_locations[function_values.index(best_function_value)]

            self.__move_particles(global_best_position, social_coefficient, random_variable_coefficient)

        return { "best_decision_variable_values": global_best_position, "best_objective_function_value": best_function_value } 

    def __initialize_particles(self, number_of_particiles):

        self.__particle_locations = []

        for individual_index in range(number_of_particiles):
            decision_variable_values = [super(MultiParticleCollisionAlgorithm, self).get_decision_variable_value_by_randomization(variable_index) for variable_index in range(self.number_of_variables)]

            self.__particle_locations.append(decision_variable_values)

    def __move_particles(self, global_best_position, social_coefficient, random_variable_coefficient):

        particle_locations_after_move = []
       
        for particle_location in self.__particle_locations:        

            decision_variable_values = []

            for variable_index in range(self.number_of_variables):

                # The value out-of-range in order to enter while loop
                new_particle_location_coordinate = self.function_wrapper.minimum_decision_variable_values()[variable_index] - 1

                while new_particle_location_coordinate < self.function_wrapper.minimum_decision_variable_values()[variable_index] or new_particle_location_coordinate > self.function_wrapper.maximum_decision_variable_values()[variable_index]:
                    new_particle_location_coordinate = (1 - social_coefficient) * particle_location[variable_index] + social_coefficient * global_best_position[variable_index] + random_variable_coefficient * (random.random() - 0.5)
                
                decision_variable_values.append(new_particle_location_coordinate)

            particle_locations_after_move.append(decision_variable_values)

        self.__particle_locations = particle_locations_after_move

    def __perturbation(self):
      print('perturbation')

    def __exploitation(self):
      print('exploitation')

    def __scattering(self):
      print('scattering')

    def __update_blackboard(self):
      print('blackboard')

In [0]:
class RosenbrookFunctionWrapper():

    def maximum_decision_variable_values(self):
        return [5, 5]

    def minimum_decision_variable_values(self):
        return [-5, -5]

    def objective_function_value(self, decision_variable_values):
        return (1 - decision_variable_values[0])**2 + 100 * (decision_variable_values[1] - decision_variable_values[0]**2)**2

    def initial_decision_variable_value_estimates(self):
        return [2, 2]


rosenbrook_function_wrapper = RosenbrookFunctionWrapper()

number_of_variables = 2
objective = "minimization"

mpca = MultiParticleCollisionAlgorithm(rosenbrook_function_wrapper, 2)

result = mpca.search()

print(result["best_decision_variable_values"][0]) # x value: Example: 1.0112
print(result["best_decision_variable_values"][1]) # y value: Example: 0.9988
print(result["best_objective_function_value"])    # f(x,y) value: Example: 0.0563  

1.022914282669731
1.0421629528340461
0.002281241601145147
